In [12]:
from functools import reduce
import pandas as pd
import re, os

from extractor import getAppData

In [13]:
app_label = "com.example.cadprodutos"
app_id = "u0a380"

# data_dir = './measurements/makeFile'
# data_dir = './measurements/localLoginSharedPref'
data_dir = './measurements/LocalLogin'

In [14]:
df = []

def deep_get(dictionary, keys, default=None):
    return reduce(lambda d, key: d.get(key, default) if isinstance(d, dict) else default, keys.split("."), dictionary)


for folder in os.scandir(data_dir):
    benchmark, iterations, filesize = folder.name.split('-')[:3]
    # benchmark, runs, file = folder.name.split('-')

    benchmark = f"{benchmark}-{iterations}"
    
    iterations = 100

    for i in range(1, iterations):
        data = getAppData(f"{folder.path}/energy-{i}.txt", app_id, log=False)

        if not data: continue

        df.append([
            # benchmark,
            # runs,
            benchmark,
            deep_get(data, 'time', default=0),
            deep_get(data, 'charge', default=0),
            deep_get(data, 'energy.total', default=0),
            deep_get(data, 'energy.cpu', default=0),
            deep_get(data, 'energy.wifi', default=0),
        ])

# df = pd.DataFrame(df, columns=['benchmark', 'runs', 'time', 'energy_total', 'energy_cpu', 'energy_wifi'])
df = pd.DataFrame(df, columns=['benchmark', 'time', 'charge', 'mAh_total', 'mAh_cpu', 'mAh_wifi'])

In [15]:
df

,benchmark,time,charge,mAh_total,mAh_cpu,mAh_wifi
0,localLoginEnc-10000,14355.0,3.129,3.990,3.990,0.00160
1,localLoginEnc-10000,27753.0,3.307,3.500,3.500,0.00130
2,localLoginEnc-10000,17372.0,3.084,3.790,3.790,0.00160
3,localLoginEnc-10000,21989.0,3.307,3.550,3.540,0.00160
4,localLoginEnc-10000,20161.0,3.084,3.790,3.790,0.00136
...,...,...,...,...,...,...
223,localLoginEnc-1000,3752.0,3.486,1.170,1.170,0.00111
224,localLoginEnc-1000,5221.0,3.486,1.010,1.010,0.00154
225,localLoginEnc-1000,3570.0,3.486,1.080,1.080,0.00111
226,localLoginEnc-1000,4152.0,3.486,1.090,1.090,0.00111


In [16]:
df['energy_total'] = df['mAh_total'] * 3.6 * df['charge']
df['energy_cpu'] = df['mAh_cpu'] * 3.6 * df['charge']
df['energy_wifi'] = df['mAh_wifi'] * 3.6 * df['charge']
df

,benchmark,time,charge,mAh_total,mAh_cpu,mAh_wifi,energy_total,energy_cpu,energy_wifi
0,localLoginEnc-10000,14355.0,3.129,3.990,3.990,0.00160,44.944956,44.944956,0.018023
1,localLoginEnc-10000,27753.0,3.307,3.500,3.500,0.00130,41.668200,41.668200,0.015477
2,localLoginEnc-10000,17372.0,3.084,3.790,3.790,0.00160,42.078096,42.078096,0.017764
3,localLoginEnc-10000,21989.0,3.307,3.550,3.540,0.00160,42.263460,42.144408,0.019048
4,localLoginEnc-10000,20161.0,3.084,3.790,3.790,0.00136,42.078096,42.078096,0.015099
...,...,...,...,...,...,...,...,...,...
223,localLoginEnc-1000,3752.0,3.486,1.170,1.170,0.00111,14.683032,14.683032,0.013930
224,localLoginEnc-1000,5221.0,3.486,1.010,1.010,0.00154,12.675096,12.675096,0.019326
225,localLoginEnc-1000,3570.0,3.486,1.080,1.080,0.00111,13.553568,13.553568,0.013930
226,localLoginEnc-1000,4152.0,3.486,1.090,1.090,0.00111,13.679064,13.679064,0.013930


In [17]:
df.to_csv('data/final.csv', index=False, header=False, mode='a')